In [11]:
import requests
import calendar
import time
import pandas as pd
import numpy as np
from datetime import datetime
import time

In [12]:
# pushshift is not up to date - you have to scrape from PRAW for updated data

# Retrieving

In [13]:
subreddits = ['bitcoin', 'ethereum']
post_url = 'https://api.pushshift.io/reddit/search/submission'
comments_url = 'https://api.pushshift.io/reddit/search/comment'


In [14]:
pd.set_option('display.max_rows', 10)

In [15]:
datetime.utcnow()

datetime.datetime(2023, 1, 13, 19, 59, 3, 504164)

In [150]:
# the secret is to not call to much, but also not ask too much in a single call, and sleeping for 10 seconds between calls

# Sometimes the requests errors out but unixtime picks up where it left off

# from /r/pushshift: "It’s the somewhat normal pushshift api being flaky thing. It might get fixed in a few days or never. 
# Please go read the sticky if you haven’t already as it has more information and context that could be useful to you."

#removed_by_category == None

def postcall(subreddit):

    #  datetime.utcnow() = 'now',                    
    # time.mktime('x'.timetuple()) = converts to unix time,                
    # and int is int
    
    unixtime = int(time.mktime(datetime.utcnow().timetuple()))
    
    # collect epochs of posts in list
    posts = []
    
    try:
    
        for i in range(200):
            print(i)

            params = {
            "subreddit": subreddit,
            'filter': 'subreddit, id, selftext, title, created_utc, author, upvote_ratio, score, user_removed, mod_removed,retrieved_utc,updated_utc,num_comments',
            "limit": 50,
            'until': unixtime
            }

            res = requests.get(post_url, params)
            
            if res.status_code == 200:

                posts.append(res.json()["data"])

                unixtime = (res.json()["data"][-1]['created_utc'])
                
            else:
                print(res.status_code)


            #throw in a sleep to fix fails
            time.sleep(5)
            
            
    except:
        print('failure', unixtime)
        print(res, res.status_code)

    return posts 

In [38]:
def commentscall(subreddit):

    #  datetime.utcnow() = 'now',                    
    # time.mktime('x'.timetuple()) = converts to unix time,                
    # and int is int
    
    unixtime = int(time.mktime(datetime.utcnow().timetuple()))
    
    # collect epochs of posts in list
    posts = []
    
    for i in range(100):
        print(i)
        
        if res.status_code == 200:
        
            params = {
            "subreddit": subreddit,
            'filter': 'subreddit, id, selftext, title, created_utc, author, upvote_ratio, score, user_removed, mod_removed',
            "limit": 50,
            'until': unixtime
            }
            
            time.sleep(5)

            res = requests.get(comments_url, params)
            print(res)

            posts.append(res.json()["data"])

            unixtime = (res.json()["data"][-1]['created_utc'])

            #throw in a sleep to fix fails
            time.sleep(5)

        else:
            print('failure', unixtime)
            print(res, res.status_code)
            

    return posts 

In [18]:
# These are the two working cells for methods above. They take a while but are pretty robust. if ungainly - takes about 45 minutes to run,
# I recommend you go do something else for a bit. batching 100 calls x2 x5 seconds sleep plus call plus try / accept logic for 524 failures.

In [ ]:
df_posts_bitcoin_list, df_posts_ethereum_list = [postcall(i) for i in subreddits]

0
1
2
3
4
5
6
7
8
9
10
524
11
12
13
524
14
15
16
17
18
19
20
21
22
23
524
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
524
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
524
88
524
89
90
91
524
92
93
94
95
96
97
98
99
100
101
102
524
103
104
105
524
106
107
524
108
109
110
111
524
112
113
114
115
116
117
118
119
120
121
524
122


In [28]:
df_comments_bitcoin_list , df_comments_ethereum_list = [commentscall(i) for i in subreddits]

0
failure 1673658788


UnboundLocalError: local variable 'res' referenced before assignment

In [74]:
# grab btc
counter = 0

for epoch in df_posts_bitcoin_list:
    
    if counter == 0:
        df_btc = pd.DataFrame(epoch)
        counter = counter + 1
    else:
        df_btc = pd.concat([df_btc,pd.DataFrame(epoch)])

df_btc


,subreddit,selftext,title,upvote_ratio,score,id,author,num_comments,created_utc,retrieved_utc,updated_utc
0,Bitcoin,,I made a tutorial showing how to use Joinmarke...,1.0,1,10b4112,benperrin117,0,1673640296,1673640307,1673640307
1,Bitcoin,,Geld verdienen,1.0,1,10b3w86,FullTechnician328,0,1673639985,1673640000,1673640000
2,Bitcoin,Bitcoin swings every cycle and I realized a lo...,I made a bot that sends me a daily text with t...,1.0,1,10b3smq,OverEducation4426,0,1673639732,1673639747,1673639748
3,Bitcoin,[https://www.btcfeargreedindextrader.com/](htt...,I made a bot that sends me a daily text with t...,1.0,1,10b3q16,OverEducation4426,0,1673639553,1673639566,1673639567
4,Bitcoin,Why are people still scared to invest when you...,when inflation has peaked the bottom was in fo...,1.0,1,10b3fr4,Accurate-Fox9427,0,1673638857,1673638868,1673638869
...,...,...,...,...,...,...,...,...,...,...,...
45,Bitcoin,,Great news,1.0,1,zl3ucb,Specific_Economist35,0,1670958195,1670958208,1670958208
46,Bitcoin,,Robinhood Could Be the Latest Casualty of FTX ...,1.0,1,zl3rof,Evil__Maid,0,1670958025,1670958036,1670958036
47,Bitcoin,,it looks like they get to keep your Bitcoin pe...,1.0,1,zl3mvl,shifuman6,0,1670957717,1670957729,1670957729
48,Bitcoin,I would like to know how/why announcements mad...,How does 1 country have so much influence over...,1.0,1,zl3m3f,unisonstero,0,1670957668,1670957680,1670957680


In [75]:
# grab ethereum
counter = 0

for epoch in df_posts_ethereum_list:
    
    if counter == 0:
        df_eth = pd.DataFrame(epoch)
        counter = counter + 1
    else:
        df_eth = pd.concat([df_eth,pd.DataFrame(epoch)])

df_eth

,subreddit,selftext,title,upvote_ratio,score,id,author,num_comments,created_utc,retrieved_utc,updated_utc
0,ethereum,[removed],Blockchain interoperability - Zelcore lets you...,1.0,1,10b4abs,zelcore_io,0,1673640915,1673640933,1673640933
1,ethereum,[removed]\n\n[View Poll](https://www.reddit.co...,Metave®se.eth chatGPT poll,1.0,1,10b2zcc,GradeOriginal,0,1673637719,1673637735,1673637736
2,ethereum,,The bankruptcy judge has permitted FTX to sell...,1.0,1,10b1vpy,Spiritual_Grand_103,0,1673635052,1673635065,1673635066
3,ethereum,,Can anyone explain me what is going on ?,1.0,1,10b121r,Weekly_Economist_898,0,1673633050,1673633067,1673633068
4,ethereum,,KZG Ceremony is live!,1.0,1,10ayu0g,AtHeartEngineer,0,1673627606,1673627616,1673627616
...,...,...,...,...,...,...,...,...,...,...,...
5,ethereum,,User review of the latest cross-chain token tr...,1.0,1,ylfr6o,Mortgage-Pitiful,0,1667511192,1667511203,1669825210
6,ethereum,"Hey fellow Redditors, I'm not sure if sharing...",Essay: Reducing the barrier for crypto using M...,1.0,1,ylek5s,whataheck,0,1667508730,1667508741,1669825224
7,ethereum,,Four Blockchains Making an Impact on the NFT M...,1.0,1,yld4jl,itsmeamirax,0,1667505745,1667505756,1669825243
8,ethereum,"\n\n**I already compare 35 exchanges**, mostl...",Hi for my staking and lending rates comparator...,1.0,1,ylcvo7,theablazefeces,0,1667505232,1667505245,1669825245


In [76]:
df_posts = pd.concat([df_btc, df_eth])

In [77]:
df_posts.shape

(8658, 11)

In [78]:
df_posts.head()

,subreddit,selftext,title,upvote_ratio,score,id,author,num_comments,created_utc,retrieved_utc,updated_utc
0,Bitcoin,,I made a tutorial showing how to use Joinmarke...,1.0,1,10b4112,benperrin117,0,1673640296,1673640307,1673640307
1,Bitcoin,,Geld verdienen,1.0,1,10b3w86,FullTechnician328,0,1673639985,1673640000,1673640000
2,Bitcoin,Bitcoin swings every cycle and I realized a lo...,I made a bot that sends me a daily text with t...,1.0,1,10b3smq,OverEducation4426,0,1673639732,1673639747,1673639748
3,Bitcoin,[https://www.btcfeargreedindextrader.com/](htt...,I made a bot that sends me a daily text with t...,1.0,1,10b3q16,OverEducation4426,0,1673639553,1673639566,1673639567
4,Bitcoin,Why are people still scared to invest when you...,when inflation has peaked the bottom was in fo...,1.0,1,10b3fr4,Accurate-Fox9427,0,1673638857,1673638868,1673638869


In [ ]:
# grab btc comments
counter = 0

for epoch in df_comments_bitcoin_list:
    
    if counter == 0:
        df_btc = pd.DataFrame(epoch)
        counter = counter + 1
    else:
        df_btc_comments = pd.concat([df_btc,pd.DataFrame(epoch)])

df_btc_comments

In [ ]:
# grab eth comments
counter = 0

for epoch in df_comments_ethereum_list:
    
    if counter == 0:
        df_eth = pd.DataFrame(epoch)
        counter = counter + 1
    else:
        df_eth_comments = pd.concat([df_eth,pd.DataFrame(epoch)])

df_eth_comments

In [ ]:
df_comments = pd.concat([df_btc_comments, df_eth_comments])

In [ ]:
df_comments

In [ ]:
df_comments.shape

# Cleaning

- Dates
- nans / [removed] / etc
- 

In [79]:
df_posts.subreddit.value_counts()

Bitcoin     4749
ethereum    3909
Name: subreddit, dtype: int64

In [80]:
df_posts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8658 entries, 0 to 9
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   subreddit      8658 non-null   object 
 1   selftext       8658 non-null   object 
 2   title          8658 non-null   object 
 3   upvote_ratio   8658 non-null   float64
 4   score          8658 non-null   int64  
 5   id             8658 non-null   object 
 6   author         8658 non-null   object 
 7   num_comments   8658 non-null   int64  
 8   created_utc    8658 non-null   int64  
 9   retrieved_utc  8658 non-null   int64  
 10  updated_utc    8658 non-null   int64  
dtypes: float64(1), int64(5), object(5)
memory usage: 811.7+ KB


In [141]:
def format(df):
    
    #get datetime
    df['created_utc'] = pd.to_datetime(df['created_utc'],unit='s')
    df['post_date'] = df['created_utc'].dt.date
    df['post_time'] = df['created_utc'].dt.time
    
    # remove obvious nans / removeds
    df = df[(df['selftext'] != '[removed]') & (df['selftext'] != '') & (df['selftext'] != "")& (df['selftext'] != "[deleted]")]
    
    # Dupes
    df = df.copy()
    df.drop_duplicates(subset=['title', 'selftext'], keep="first", inplace = True)
    
    return df



In [142]:
df_posts = format(df_posts)

In [143]:
for i in [df_comments, df_posts]:
    df_comments_cleaned = removed(df_comments)

NameError: name 'df_comments' is not defined

In [144]:
df_posts.subreddit.value_counts()

Bitcoin     1529
ethereum     739
Name: subreddit, dtype: int64

In [145]:
#df_comments.subreddit.value_counts()

In [146]:
pd.merge(df_posts, df_comments)

NameError: name 'df_comments' is not defined

# EDA

# Questions

1. Ads? 
2. Removed? Removed text?
3. How many are text vs. content?
4. Authorship
5. Title keywords
6. Text keywords
7. Title / text relationship
8. Price / timeseries

- Compare old time (pre-popular) subreddit to current


In [147]:
df_posts.to_csv('./data/source.csv', index = False)

In [149]:
df_posts.selftext.value_counts()
# hyperlink will get removed, the only other duplicate is blockstream spam

https://www.coinwetalk.com/mining/partner/103294                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        